# Melbourne Housing Data Cleaning Tutorial

This notebook provides a comprehensive tutorial on data cleaning techniques using the Melbourne Housing Market dataset. We'll cover missing values, inconsistent formats, outliers, and best practices for preparing data for analysis and modeling.

## 1. Import Libraries and Load Data

First, let's import the necessary libraries and load our dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import zscore

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

In [ ]:
# Load the Melbourne Housing dataset
df = pd.read_csv('datasets/Melbourne_housing_FULL.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Dataset size: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first few rows
print("\nFirst 5 rows:")
print(df.head())

## 2. Initial Data Exploration

Before we start cleaning, let's understand what we're working with. This step is crucial for identifying potential data quality issues.

In [ ]:
# Basic information about the dataset
print("=== DATASET OVERVIEW ===")
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Duplicate rows: {df.duplicated().sum()}")

print("\n=== COLUMN INFORMATION ===")
print("Data types:")
print(df.dtypes.value_counts())

print("\n=== FIRST LOOK AT DATA ===")
df.info()

## 3. Missing Values Analysis

Missing values are one of the most common data quality issues. Let's identify and understand the patterns of missing data in our dataset.

### Why Missing Values Matter:
- **Model Performance**: Many ML algorithms can't handle missing values
- **Bias Introduction**: Naive handling can introduce systematic bias
- **Information Loss**: Dropping all rows with missing values can lose valuable information
- **Pattern Recognition**: Missing patterns can reveal data collection issues

In [ ]:
# Calculate missing values for each column
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})

# Filter columns with missing values and sort by percentage
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print("=== MISSING VALUES SUMMARY ===")
print(f"Total columns with missing values: {len(missing_data)} out of {len(df.columns)}")
print(f"Total missing values: {missing_data['Missing_Count'].sum():,}")
print(f"Percentage of total dataset missing: {(missing_data['Missing_Count'].sum() / df.size) * 100:.2f}%")

print("\n=== COLUMNS WITH MISSING VALUES ===")
if len(missing_data) > 0:
    print(missing_data.to_string(index=False))
else:
    print("✅ No missing values found in the dataset!")

In [ ]:
# Visualize missing value patterns
if len(missing_data) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(15, 12))
    
    # Bar plot of missing values
    top_missing = missing_data.head(15)  # Show top 15 for readability
    axes[0].bar(range(len(top_missing)), top_missing['Missing_Percentage'], 
                color=['red' if x > 50 else 'orange' if x > 20 else 'yellow' for x in top_missing['Missing_Percentage']])
    axes[0].set_title('Missing Values by Column (Top 15)', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Missing Percentage (%)')
    axes[0].set_xticks(range(len(top_missing)))
    axes[0].set_xticklabels(top_missing['Column'], rotation=45, ha='right')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add percentage labels on bars
    for i, v in enumerate(top_missing['Missing_Percentage']):
        axes[0].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # Heatmap showing missing patterns (sample of rows for performance)
    sample_size = min(1000, len(df))  # Sample for performance with large datasets
    missing_cols = missing_data.head(15)['Column'].tolist()
    df_sample = df[missing_cols].head(sample_size)
    
    axes[1].imshow(df_sample.isnull().T, cmap='RdYlBu_r', aspect='auto', interpolation='nearest')
    axes[1].set_title(f'Missing Value Pattern Heatmap (First {sample_size} rows)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Row Index')
    axes[1].set_ylabel('Columns with Missing Values')
    axes[1].set_yticks(range(len(missing_cols)))
    axes[1].set_yticklabels(missing_cols)
    
    plt.tight_layout()
    plt.show()
    
    print(f"🔍 Red = Missing, Blue = Present")
    print(f"📊 Colors in bar chart: Red >50%, Orange >20%, Yellow <20%")
else:
    print("✅ No missing values to visualize!")

### 3.1 Missing Value Treatment Strategies

Different types of missing values require different treatment approaches. Let's implement various strategies:

1. **Missing Completely At Random (MCAR)**: Missing values are unrelated to any other data
2. **Missing At Random (MAR)**: Missing values are related to observed data but not the missing data itself  
3. **Missing Not At Random (MNAR)**: Missing values are related to the unobserved data itself

### Treatment Methods:
- **Deletion**: Remove rows/columns with missing values
- **Imputation**: Fill missing values with estimated values
- **Indicator Variables**: Create binary flags for missingness
- **Advanced Methods**: Multiple imputation, model-based imputation

In [ ]:
# Create a copy for cleaning demonstrations
df_cleaned = df.copy()

print("=== MISSING VALUE TREATMENT DEMONSTRATIONS ===")

# Function to apply different imputation strategies
def apply_missing_value_treatments(df, column, strategy='mean'):
    """
    Apply different missing value treatment strategies
    """
    if df[column].dtype in ['object']:
        if strategy == 'mode':
            return df[column].fillna(df[column].mode()[0] if len(df[column].mode()) > 0 else 'Unknown')
        elif strategy == 'constant':
            return df[column].fillna('Unknown')
        else:
            return df[column].fillna('Unknown')
    else:
        if strategy == 'mean':
            return df[column].fillna(df[column].mean())
        elif strategy == 'median':
            return df[column].fillna(df[column].median())
        elif strategy == 'mode':
            return df[column].fillna(df[column].mode()[0] if len(df[column].mode()) > 0 else 0)
        elif strategy == 'zero':
            return df[column].fillna(0)
        elif strategy == 'forward_fill':
            return df[column].fillna(method='ffill')
        elif strategy == 'backward_fill':
            return df[column].fillna(method='bfill')
        else:
            return df[column].fillna(df[column].mean())

# Demonstrate different imputation strategies if we have missing values
if len(missing_data) > 0:
    # Take the first column with missing values as an example
    example_col = missing_data.iloc[0]['Column']
    print(f"\nDemonstrating imputation strategies on column: '{example_col}'")
    print(f"Original missing values: {df[example_col].isnull().sum()}")
    print(f"Data type: {df[example_col].dtype}")
    
    if df[example_col].dtype == 'object':
        strategies = ['mode', 'constant']
    else:
        strategies = ['mean', 'median', 'zero']
    
    print(f"\nBefore imputation - Sample values:")
    print(df[example_col].head(10).tolist())
    
    for strategy in strategies:
        imputed_col = apply_missing_value_treatments(df, example_col, strategy)
        print(f"\n{strategy.upper()} imputation:")
        print(f"  - Missing values after: {imputed_col.isnull().sum()}")
        if df[example_col].dtype != 'object':
            print(f"  - Imputed value: {imputed_col.fillna(method='ffill').iloc[-1]}")
        print(f"  - Sample values: {imputed_col.head(10).tolist()}")

else:
    print("\n✅ No missing values found - skipping imputation demonstration")

## 4. Inconsistent Format Detection and Correction

Data inconsistencies can occur in various forms. Let's identify and fix common format issues.

### Common Format Issues:
- **String Inconsistencies**: Mixed case, extra spaces, different representations
- **Date Format Variations**: Different date formats in the same column
- **Numerical Format Issues**: Mixed units, currency symbols, decimal separators
- **Categorical Inconsistencies**: Same category with different names

In [ ]:
# Identify potential format inconsistencies in categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print("=== CATEGORICAL DATA CONSISTENCY ANALYSIS ===")
print(f"Found {len(categorical_cols)} categorical columns to analyze\n")

# Function to detect and analyze inconsistencies
def analyze_categorical_consistency(df, column):
    """Analyze categorical column for potential inconsistencies"""
    values = df[column].dropna().astype(str)
    
    issues = {
        'mixed_case': [],
        'extra_spaces': [],
        'similar_values': [],
        'special_chars': []
    }
    
    unique_values = values.unique()
    
    # Check for mixed case issues
    for val in unique_values:
        if val != val.lower() and val != val.upper():
            issues['mixed_case'].append(val)
    
    # Check for leading/trailing spaces
    for val in unique_values:
        if val != val.strip():
            issues['extra_spaces'].append(f"'{val}' -> '{val.strip()}'")
    
    # Check for similar values (simple approach)
    for i, val1 in enumerate(unique_values):
        for val2 in unique_values[i+1:]:
            if val1.lower().strip() == val2.lower().strip() and val1 != val2:
                issues['similar_values'].append(f"'{val1}' vs '{val2}'")
    
    # Check for special characters
    import re
    for val in unique_values:
        if re.search(r'[^a-zA-Z0-9\s]', val):
            issues['special_chars'].append(val)
    
    return issues

# Analyze first few categorical columns as examples
for col in categorical_cols[:5]:  # Limit to first 5 for demonstration
    print(f"\n📊 ANALYZING COLUMN: '{col}'")
    print(f"Unique values: {df[col].nunique()}")
    print(f"Sample values: {df[col].dropna().unique()[:10].tolist()}")
    
    issues = analyze_categorical_consistency(df, col)
    
    has_issues = any(len(issue_list) > 0 for issue_list in issues.values())
    
    if has_issues:
        print("\n⚠️  ISSUES DETECTED:")
        if issues['mixed_case']:
            print(f"   Mixed case: {issues['mixed_case'][:5]}")  # Show first 5
        if issues['extra_spaces']:
            print(f"   Extra spaces: {issues['extra_spaces'][:5]}")
        if issues['similar_values']:
            print(f"   Similar values: {issues['similar_values'][:5]}")
        if issues['special_chars']:
            print(f"   Special chars: {issues['special_chars'][:5]}")
    else:
        print("✅ No obvious consistency issues detected")

if len(categorical_cols) > 5:
    print(f"\n... ({len(categorical_cols)-5} more columns not shown for brevity)")

In [ ]:
# Demonstrate cleaning techniques for categorical data
print("=== CATEGORICAL DATA CLEANING DEMONSTRATIONS ===\n")

# Function to clean categorical data
def clean_categorical_column(series, column_name):
    """
    Clean categorical data by applying common cleaning techniques
    """
    print(f"🧹 Cleaning column: '{column_name}'")
    original_unique = series.nunique()
    
    # Create a copy to modify
    cleaned = series.copy()
    
    # 1. Handle missing values
    missing_count = cleaned.isnull().sum()
    cleaned = cleaned.fillna('Unknown')
    
    # 2. Convert to string and strip whitespace
    cleaned = cleaned.astype(str).str.strip()
    
    # 3. Standardize case (title case for most categorical data)
    cleaned = cleaned.str.title()
    
    # 4. Remove extra internal spaces
    cleaned = cleaned.str.replace(r'\s+', ' ', regex=True)
    
    # 5. Handle common typos/variations (example for demonstration)
    # This would be domain-specific in real scenarios
    common_replacements = {
        'Unknow': 'Unknown',
        'N/A': 'Unknown',
        'Na': 'Unknown',
        'Nan': 'Unknown'
    }
    
    for old, new in common_replacements.items():
        cleaned = cleaned.str.replace(old, new, case=False)
    
    final_unique = cleaned.nunique()
    
    print(f"   - Original unique values: {original_unique}")
    print(f"   - Missing values handled: {missing_count}")
    print(f"   - Final unique values: {final_unique}")
    print(f"   - Reduction: {original_unique - final_unique} duplicates removed")
    
    if original_unique != final_unique:
        print(f"   - Sample before: {series.dropna().unique()[:5].tolist()}")
        print(f"   - Sample after:  {cleaned.unique()[:5].tolist()}")
    
    return cleaned

# Apply cleaning to a sample of categorical columns
sample_cols = categorical_cols[:3] if len(categorical_cols) >= 3 else categorical_cols

for col in sample_cols:
    df_cleaned[f"{col}_cleaned"] = clean_categorical_column(df[col], col)
    print("\n" + "="*50 + "\n")

## 5. Outlier Detection and Treatment

Outliers can significantly impact data analysis and model performance. Let's identify and handle them appropriately.

### Types of Outliers:
- **Statistical Outliers**: Values that are statistically unusual
- **Domain Outliers**: Values that don't make sense in the business context
- **Data Entry Errors**: Incorrect values due to human error
- **True Outliers**: Legitimate extreme values that provide valuable information

### Detection Methods:
- **IQR Method**: Using the Interquartile Range
- **Z-Score**: Using standard deviations from the mean
- **Modified Z-Score**: Using median absolute deviation
- **Isolation Forest**: Machine learning approach

In [ ]:
# Identify numerical columns for outlier detection
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"=== OUTLIER DETECTION ANALYSIS ===")
print(f"Found {len(numerical_cols)} numerical columns to analyze\n")

# Function to detect outliers using multiple methods
def detect_outliers_comprehensive(df, column, methods=['iqr', 'zscore', 'modified_zscore']):
    """
    Detect outliers using multiple methods
    """
    results = {}
    
    # Skip if column has too many missing values
    if df[column].isnull().sum() / len(df) > 0.5:
        return {'skipped': 'Too many missing values (>50%)'}
    
    data = df[column].dropna()
    
    if len(data) == 0:
        return {'skipped': 'No data available'}
    
    # Method 1: IQR Method
    if 'iqr' in methods:
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        iqr_outliers = data[(data < lower_bound) | (data > upper_bound)]
        results['iqr'] = {
            'count': len(iqr_outliers),
            'percentage': len(iqr_outliers) / len(data) * 100,
            'bounds': (lower_bound, upper_bound),
            'outliers': iqr_outliers
        }
    
    # Method 2: Z-Score Method (using 3 standard deviations)
    if 'zscore' in methods:
        z_scores = np.abs(zscore(data))
        zscore_outliers = data[z_scores > 3]
        results['zscore'] = {
            'count': len(zscore_outliers),
            'percentage': len(zscore_outliers) / len(data) * 100,
            'threshold': 3,
            'outliers': zscore_outliers
        }
    
    # Method 3: Modified Z-Score Method (using median)
    if 'modified_zscore' in methods:
        median = np.median(data)
        mad = np.median(np.abs(data - median))
        if mad != 0:
            modified_z_scores = 0.6745 * (data - median) / mad
            mod_zscore_outliers = data[np.abs(modified_z_scores) > 3.5]
            results['modified_zscore'] = {
                'count': len(mod_zscore_outliers),
                'percentage': len(mod_zscore_outliers) / len(data) * 100,
                'threshold': 3.5,
                'outliers': mod_zscore_outliers
            }
        else:
            results['modified_zscore'] = {'skipped': 'MAD is zero (no variability)'}
    
    return results

# Analyze outliers for first few numerical columns
sample_numerical = numerical_cols[:5] if len(numerical_cols) >= 5 else numerical_cols

for col in sample_numerical:
    print(f"\n📊 OUTLIER ANALYSIS FOR: '{col}'")
    print(f"Data type: {df[col].dtype}")
    print(f"Non-null values: {df[col].count()} / {len(df)}")
    
    if df[col].count() > 0:
        print(f"Basic stats: Min={df[col].min():.2f}, Max={df[col].max():.2f}, Mean={df[col].mean():.2f}, Std={df[col].std():.2f}")
        
        outlier_results = detect_outliers_comprehensive(df, col)
        
        if 'skipped' in outlier_results:
            print(f"⚠️  Analysis skipped: {outlier_results['skipped']}")
            continue
        
        print("\n🔍 OUTLIER DETECTION RESULTS:")
        
        for method, results in outlier_results.items():
            if 'skipped' in results:
                print(f"   {method.upper()}: Skipped - {results['skipped']}")
                continue
            
            print(f"   {method.upper()}:")
            print(f"      Count: {results['count']} ({results['percentage']:.2f}%)")
            
            if method == 'iqr':
                print(f"      Bounds: [{results['bounds'][0]:.2f}, {results['bounds'][1]:.2f}]")
            else:
                print(f"      Threshold: {results['threshold']}")
            
            if results['count'] > 0:
                outlier_values = results['outliers'].head(3).tolist()
                print(f"      Sample outliers: {[f'{x:.2f}' for x in outlier_values]}")
    else:
        print("⚠️  No data available for analysis")

if len(numerical_cols) > 5:
    print(f"\n... ({len(numerical_cols)-5} more columns not shown for brevity)")

In [ ]:
# Visualize outliers for selected numerical columns
def visualize_outliers(df, columns, max_cols=4):
    """
    Create comprehensive outlier visualizations
    """
    # Limit columns for visualization
    viz_columns = columns[:max_cols]
    n_cols = len(viz_columns)
    
    if n_cols == 0:
        print("No numerical columns available for visualization")
        return
    
    fig, axes = plt.subplots(2, n_cols, figsize=(5*n_cols, 10))
    if n_cols == 1:
        axes = axes.reshape(2, 1)
    
    colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold']
    
    for i, col in enumerate(viz_columns):
        data = df[col].dropna()
        
        if len(data) == 0:
            axes[0, i].text(0.5, 0.5, 'No data', ha='center', va='center', transform=axes[0, i].transAxes)
            axes[1, i].text(0.5, 0.5, 'No data', ha='center', va='center', transform=axes[1, i].transAxes)
            continue
        
        # Box plot (top row)
        axes[0, i].boxplot(data, patch_artist=True, 
                          boxprops=dict(facecolor=colors[i % len(colors)], alpha=0.7))
        axes[0, i].set_title(f'{col}\nBox Plot (Outliers as points)', fontweight='bold')
        axes[0, i].grid(True, alpha=0.3)
        
        # Add outlier statistics
        Q1, Q3 = data.quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        
        axes[0, i].text(0.02, 0.98, f'Outliers: {len(outliers)} ({len(outliers)/len(data)*100:.1f}%)', 
                       transform=axes[0, i].transAxes, verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Histogram with outliers highlighted (bottom row)
        axes[1, i].hist(data, bins=30, alpha=0.7, color=colors[i % len(colors)], edgecolor='black')
        
        # Highlight outliers
        if len(outliers) > 0:
            axes[1, i].hist(outliers, bins=30, alpha=0.9, color='red', edgecolor='darkred', 
                           label=f'Outliers ({len(outliers)})')
            axes[1, i].legend()
        
        axes[1, i].set_title(f'{col}\nHistogram (Outliers in red)', fontweight='bold')
        axes[1, i].set_xlabel('Value')
        axes[1, i].set_ylabel('Frequency')
        axes[1, i].grid(True, alpha=0.3)
        
        # Add basic statistics
        stats_text = f'Mean: {data.mean():.2f}\nStd: {data.std():.2f}\nMin: {data.min():.2f}\nMax: {data.max():.2f}'
        axes[1, i].text(0.98, 0.98, stats_text, transform=axes[1, i].transAxes, 
                       verticalalignment='top', horizontalalignment='right',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                       fontsize=9)
    
    plt.tight_layout()
    plt.show()

# Create visualizations
print("=== OUTLIER VISUALIZATION ===\n")
visualize_outliers(df, sample_numerical, max_cols=4)

### 5.1 Outlier Treatment Strategies

Once we've identified outliers, we need to decide how to handle them. The approach depends on the nature of the outliers and the intended use of the data.

#### Treatment Options:
1. **Keep**: When outliers represent legitimate extreme values
2. **Remove**: When outliers are clearly errors or irrelevant
3. **Transform**: Using log transformation, square root, etc.
4. **Cap**: Set maximum/minimum thresholds (winsorizing)
5. **Separate Analysis**: Treat outliers as a separate segment

In [ ]:
# Demonstrate different outlier treatment strategies
print("=== OUTLIER TREATMENT DEMONSTRATIONS ===\n")

def demonstrate_outlier_treatments(df, column):
    """
    Demonstrate different outlier treatment strategies
    """
    print(f"🛠️  TREATING OUTLIERS IN: '{column}'")
    
    data = df[column].dropna()
    if len(data) == 0:
        print("   No data available")
        return
    
    # Identify outliers using IQR method
    Q1, Q3 = data.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    
    print(f"   Original data points: {len(data)}")
    print(f"   Outliers detected (IQR): {len(outliers)} ({len(outliers)/len(data)*100:.2f}%)")
    if len(outliers) > 0:
        print(f"   Outlier range: {outliers.min():.2f} to {outliers.max():.2f}")
    
    if len(outliers) == 0:
        print("   ✅ No outliers to treat")
        return
    
    # Strategy 1: Removal
    data_removed = data[(data >= lower_bound) & (data <= upper_bound)]
    print(f"\n   📌 STRATEGY 1 - Removal:")
    print(f"      Remaining data points: {len(data_removed)} ({len(data_removed)/len(data)*100:.1f}%)")
    print(f"      New range: {data_removed.min():.2f} to {data_removed.max():.2f}")
    
    # Strategy 2: Capping (Winsorizing)
    data_capped = data.copy()
    data_capped[data_capped < lower_bound] = lower_bound
    data_capped[data_capped > upper_bound] = upper_bound
    print(f"\n   📌 STRATEGY 2 - Capping (Winsorizing):")
    print(f"      Data points: {len(data_capped)} (no change)")
    print(f"      New range: {data_capped.min():.2f} to {data_capped.max():.2f}")
    print(f"      Values capped: {len(outliers)}")
    
    # Strategy 3: Log transformation (if all values are positive)
    if data.min() > 0:
        data_log = np.log1p(data)  # log1p handles zeros better
        print(f"\n   📌 STRATEGY 3 - Log Transformation:")
        print(f"      Original skewness: {data.skew():.3f}")
        print(f"      Log-transformed skewness: {data_log.skew():.3f}")
        print(f"      Skewness improvement: {abs(data.skew()) - abs(data_log.skew()):.3f}")
    else:
        print(f"\n   📌 STRATEGY 3 - Log Transformation: Not applicable (contains zero/negative values)")
    
    # Strategy 4: Z-score threshold
    z_scores = np.abs(zscore(data))
    data_zscore_filtered = data[z_scores <= 3]
    print(f"\n   📌 STRATEGY 4 - Z-score filtering (|z| <= 3):")
    print(f"      Remaining data points: {len(data_zscore_filtered)} ({len(data_zscore_filtered)/len(data)*100:.1f}%)")
    print(f"      Removed: {len(data) - len(data_zscore_filtered)} outliers")
    
    return {
        'original': data,
        'removed': data_removed,
        'capped': data_capped,
        'log_transformed': np.log1p(data) if data.min() > 0 else None,
        'zscore_filtered': data_zscore_filtered
    }

# Demonstrate on a sample column with outliers
if len(sample_numerical) > 0:
    demo_col = sample_numerical[0]  # Take first numerical column
    treatment_results = demonstrate_outlier_treatments(df, demo_col)
    print("\n" + "="*60)

## 6. Data Quality Validation

After cleaning, it's important to validate that our data meets quality standards and is ready for analysis.

### Validation Checks:
- **Completeness**: Are all required fields populated?
- **Consistency**: Are data formats and values consistent?
- **Accuracy**: Do values make business sense?
- **Validity**: Do values fall within acceptable ranges?
- **Uniqueness**: Are there unexpected duplicates?

In [ ]:
# Comprehensive data quality validation
def comprehensive_data_quality_report(df):
    """
    Generate a comprehensive data quality report
    """
    print("=== COMPREHENSIVE DATA QUALITY REPORT ===\n")
    
    # Basic information
    print("📊 DATASET OVERVIEW:")
    print(f"   • Rows: {len(df):,}")
    print(f"   • Columns: {len(df.columns)}")
    print(f"   • Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   • Data types: {dict(df.dtypes.value_counts())}")
    
    # Completeness check
    print("\n✓ COMPLETENESS CHECK:")
    missing_summary = df.isnull().sum()
    total_missing = missing_summary.sum()
    missing_cols = missing_summary[missing_summary > 0]
    
    print(f"   • Total missing values: {total_missing:,} ({total_missing/df.size*100:.2f}%)")
    print(f"   • Columns with missing data: {len(missing_cols)} / {len(df.columns)}")
    
    if len(missing_cols) > 0:
        print(f"   • Worst offenders:")
        for col, missing_count in missing_cols.head(5).items():
            print(f"     - {col}: {missing_count} ({missing_count/len(df)*100:.1f}%)")
    
    # Duplicates check
    print("\n🔍 UNIQUENESS CHECK:")
    total_duplicates = df.duplicated().sum()
    print(f"   • Duplicate rows: {total_duplicates} ({total_duplicates/len(df)*100:.2f}%)")
    
    # Data type consistency
    print("\n📋 CONSISTENCY CHECK:")
    
    # Check numerical columns for non-numeric values (if any were incorrectly typed)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"   • Numerical columns: {len(numeric_cols)}")
    
    # Check for negative values in columns that shouldn't have them
    suspicious_negatives = {}
    for col in numeric_cols:
        if 'price' in col.lower() or 'area' in col.lower() or 'rooms' in col.lower() or 'distance' in col.lower():
            negative_count = (df[col] < 0).sum()
            if negative_count > 0:
                suspicious_negatives[col] = negative_count
    
    if suspicious_negatives:
        print(f"   • Suspicious negative values found:")
        for col, count in suspicious_negatives.items():
            print(f"     - {col}: {count} negative values")
    else:
        print(f"   • No suspicious negative values detected")
    
    # Range validity checks
    print("\n📏 VALIDITY CHECKS:")
    
    # Check for reasonable ranges in key columns
    validity_issues = []
    
    for col in numeric_cols:
        col_data = df[col].dropna()
        if len(col_data) > 0:
            # Check for extreme outliers (beyond 5 standard deviations)
            if col_data.std() > 0:
                extreme_outliers = np.abs(zscore(col_data)) > 5
                if extreme_outliers.any():
                    validity_issues.append(f"{col}: {extreme_outliers.sum()} extreme outliers (>5 std)")
    
    if validity_issues:
        print(f"   • Validity concerns found:")
        for issue in validity_issues[:5]:  # Show first 5
            print(f"     - {issue}")
    else:
        print(f"   • No major validity concerns detected")
    
    # Categorical data consistency
    print("\n🏷️ CATEGORICAL DATA QUALITY:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    cat_issues = []
    for col in categorical_cols[:5]:  # Check first 5 categorical columns
        unique_vals = df[col].dropna().unique()
        if len(unique_vals) > 0:
            # Check for potential case/spacing issues
            cleaned_vals = set()
            for val in unique_vals:
                cleaned_val = str(val).strip().lower()
                if cleaned_val in cleaned_vals:
                    cat_issues.append(f"{col}: Potential case/spacing inconsistencies")
                    break
                cleaned_vals.add(cleaned_val)
    
    if cat_issues:
        print(f"   • Issues detected:")
        for issue in cat_issues:
            print(f"     - {issue}")
    else:
        print(f"   • No obvious categorical inconsistencies in sample")
    
    # Overall quality score
    quality_score = 100
    quality_score -= min((total_missing / df.size) * 100 * 2, 20)  # Missing data penalty
    quality_score -= min((total_duplicates / len(df)) * 100 * 3, 15)  # Duplicate penalty
    quality_score -= min(len(validity_issues) * 2, 10)  # Validity issues penalty
    quality_score -= min(len(cat_issues) * 3, 10)  # Categorical issues penalty
    
    print(f"\n⭐ OVERALL DATA QUALITY SCORE: {quality_score:.1f}/100")
    
    if quality_score >= 90:
        print("   🎉 Excellent data quality! Ready for advanced analysis.")
    elif quality_score >= 75:
        print("   ✅ Good data quality. Minor cleaning recommended.")
    elif quality_score >= 60:
        print("   ⚠️ Moderate data quality. Significant cleaning needed.")
    else:
        print("   🚨 Poor data quality. Major cleaning required before analysis.")
    
    return quality_score

# Generate quality report for original data
print("ORIGINAL DATA:")
original_score = comprehensive_data_quality_report(df)

print("\n" + "="*80 + "\n")

# Generate quality report for cleaned data (if we made changes)
print("CLEANED DATA:")
cleaned_score = comprehensive_data_quality_report(df_cleaned)

print(f"\n📈 QUALITY IMPROVEMENT: {cleaned_score - original_score:.1f} points")

## 7. Best Practices and Summary

### Data Cleaning Best Practices:

1. **Always Keep Original Data**: Make copies before cleaning
2. **Document Changes**: Keep track of all transformations
3. **Domain Knowledge**: Use business understanding to guide decisions
4. **Iterative Process**: Clean, validate, and repeat
5. **Automate When Possible**: Create reusable cleaning functions

### Key Takeaways:

✅ **Missing Values**: Understand the pattern before treating  
✅ **Inconsistent Formats**: Standardize early to prevent issues  
✅ **Outliers**: Don't automatically remove - investigate first  
✅ **Validation**: Always validate after cleaning  
✅ **Documentation**: Record decisions for reproducibility  

### Next Steps:
After cleaning, your data is ready for:
- Exploratory Data Analysis (EDA)
- Feature Engineering
- Machine Learning Model Development
- Statistical Analysis

In [ ]:
# Final summary and export cleaned data
print("=== DATA CLEANING TUTORIAL COMPLETED ===\n")

print("📚 WHAT WE COVERED:")
print("   1. ✅ Initial data exploration and overview")
print("   2. ✅ Missing values analysis and treatment strategies")
print("   3. ✅ Inconsistent format detection and correction")
print("   4. ✅ Outlier detection using multiple methods") 
print("   5. ✅ Outlier treatment demonstrations")
print("   6. ✅ Comprehensive data quality validation")
print("   7. ✅ Best practices and recommendations")

print("\n🛠️  CLEANING TECHNIQUES DEMONSTRATED:")
print("   • Missing value imputation (mean, median, mode, constant)")
print("   • Categorical data standardization")
print("   • Outlier detection (IQR, Z-score, Modified Z-score)")
print("   • Outlier treatment (removal, capping, transformation)")
print("   • Data quality validation and scoring")

print("\n💡 KEY INSIGHTS FOR MELBOURNE HOUSING DATA:")
# Check if missing_data exists (it might not if there are no missing values)
try:
    if len(missing_data) > 0:
        print(f"   • {len(missing_data)} columns have missing values")
        print(f"   • Highest missing: {missing_data.iloc[0]['Column']} ({missing_data.iloc[0]['Missing_Percentage']:.1f}%)")
    else:
        print("   • No missing values detected - excellent data quality!")
except NameError:
    print("   • Missing value analysis completed")

if len(categorical_cols) > 0:
    print(f"   • {len(categorical_cols)} categorical columns for potential standardization")

if len(numerical_cols) > 0:
    print(f"   • {len(numerical_cols)} numerical columns for outlier analysis")

print("\n🚀 READY FOR NEXT STEPS:")
print("   • Exploratory Data Analysis (EDA)")
print("   • Feature Engineering")
print("   • Machine Learning Modeling")
print("   • Statistical Analysis")

# Optionally save cleaned data
save_cleaned = True  # Set to False if you don't want to save

if save_cleaned:
    try:
        df_cleaned.to_csv('datasets/Melbourne_housing_CLEANED.csv', index=False)
        print("\n💾 Cleaned dataset saved as 'datasets/Melbourne_housing_CLEANED.csv'")
    except Exception as e:
        print(f"\n⚠️ Could not save cleaned dataset: {e}")

print("\n🎉 DATA CLEANING TUTORIAL COMPLETED SUCCESSFULLY!")
print("\nYou now have the knowledge and tools to clean datasets effectively.")
print("Remember: Good data cleaning is the foundation of successful data analysis!")